In [24]:
import pandas as pd

### To do 
 
- Garder seulement les communes, ou traiter les différentes nomenclatures fonctionnelles (régions et dept)
- retraitement par fonction
- Améliorer le cleaning

In [139]:
DATA_DIR = 'data_input/'
DATA_FILE = 'BalanceSPL_Fonction_2013_Juin2019.csv'

In [140]:
data = pd.read_csv(DATA_DIR + DATA_FILE, sep=';', encoding = "ISO-8859-1", dtype=str)

In [141]:
data = data[data['CATEG'] == 'Commune']

In [78]:
COLUMNS = ['IDENT', 'NDEPT', 'INSEE', 'siren', 'LBUDG', 'COMPTE', 'OOBDEB', 'OOBCRE', 
           'FONCTION', 'SD', 'SC', 'OBNETDEB', 'OBNETCRE']

In [93]:
from importlib import reload
import lib
reload(lib)
from lib import string_to_float, get_code_aggregat, label_aggregat, label_fonction, pad_ndept, clean_insee_type

In [94]:
# convertir colonnes de dépense en float
for col in ['SD', 'SC', 'OBNETDEB', 'OBNETCRE', 'OOBDEB', 'OOBCRE']:
    data[col] = data[col].apply(string_to_float)

In [95]:
# nettoyer les types insee
data['INSEE'] = data['INSEE'].apply(clean_insee_type)

In [41]:
# étiquetter les aggregats
data['code_aggregat'] = data['COMPTE'].apply(get_code_aggregat)
data = data.dropna(subset=['code_aggregat']).reset_index()
data['aggregat'] = data['code_aggregat'].apply(label_aggregat)

In [42]:
# calculer les dépenses selon les aggrégats
calcul_depense_1 = data['code_aggregat'].isin([1.0, 2.0, 3.0, 4.0, 5.0, 6.0])
calcul_depense_2 = data['code_aggregat'].isin([7.0, 8.0, 9.0])
calcul_depense_3 = data['code_aggregat'].isin([10.0, 11.0])

data.loc[calcul_depense_1, 'depense'] = data['SD'] - data['SC']
data.loc[calcul_depense_2, 'depense'] = data['OBNETDEB'] - data['OBNETCRE'] - (data['OOBDEB'] - data['OOBCRE']) 
data.loc[calcul_depense_3, 'depense'] = data['OBNETDEB']

In [ ]:
data['FONCTION_type'] = data['FONCTION_type']

In [151]:
def clean_fonction_code(fonction):
    return fonction.replace('-', '').replace('.', '')

In [121]:
from functools import reduce
import numpy as np

In [152]:
def max_length(series):
    return reduce(lambda x, y: max(x, y), map(len, series))

In [233]:
def to_set(series):
    return set

In [142]:
data['fonc'] = data['FONCTION'].fillna(0).apply(str).apply(clean_fonction_code)

In [230]:
data[data['LBUDG'] == 'MONTPELLIER']

,EXER,IDENT,NDEPT,LBUDG,INSEE,CBUDG,CTYPE,CSTYP,NOMEN,siren,...,OBNETDEB,OBNETCRE,ONBDEB,ONBCRE,OOBDEB,OOBCRE,SD,SC,fonc,fonc_1
1184991,2013,21340172201787,034,MONTPELLIER,172,1,101,00,M14,213401722,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","282518520,63",0,0
1184992,2013,21340172201787,034,MONTPELLIER,172,1,101,00,M14,213401722,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","210747406,97",0,0
1184993,2013,21340172201787,034,MONTPELLIER,172,1,101,00,M14,213401722,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","23506366,83",0,0
1184994,2013,21340172201787,034,MONTPELLIER,172,1,101,00,M14,213401722,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","12025076,44",0,0
1184995,2013,21340172201787,034,MONTPELLIER,172,1,101,00,M14,213401722,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","112093,62",0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1186645,2013,21340172201787,034,MONTPELLIER,172,1,101,00,M14,213401722,...,"19651686,86","0,00","0,00","0,00","19651686,86","0,00","19651686,86","0,00",934,9
1186646,2013,21340172201787,034,MONTPELLIER,172,1,101,00,M14,213401722,...,"0,00","173636,78","0,00","0,00","0,00","173636,78","0,00","173636,78",934,9
1186647,2013,21340172201787,034,MONTPELLIER,172,1,101,00,M14,213401722,...,"0,00","1001371,47","0,00","0,00","0,00","1001371,47","0,00","1001371,47",934,9
1186648,2013,21340172201787,034,MONTPELLIER,172,1,101,00,M14,213401722,...,"74619,94","0,00","0,00","0,00","0,00","0,00","74619,94","0,00",936,9


In [203]:
data['fonc_1'] = data['fonc'].apply(lambda x: x[0]).apply(int)

In [213]:
grouped[('fonc_1', 'amax')][5061]

9

In [237]:
grouped = (
    data
    .groupby('IDENT')
    .agg({'fonc': max_length, 'fonc_1': lambda x: set(x)})
    .reset_index()
)

In [248]:
grouped

,IDENT,fonc,fonc_1
0,20000871200016,3,"{0, 8, 2}"
1,20000872000019,3,"{0, 2, 3, 5, 8}"
2,20000873800011,3,"{0, 1, 2, 8}"
3,20000874600014,3,"{0, 1, 2, 3, 4, 8}"
4,20000875300010,3,"{0, 1, 2, 3, 4, 5, 6, 8}"
...,...,...,...
5058,21974020600012,4,"{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}"
5059,21974021400016,3,"{0, 1, 2, 3, 4, 5, 7, 8, 9}"
5060,21974022200019,3,"{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}"
5061,21974023000012,3,"{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}"


In [243]:
conditions = (
    True
    & (grouped['fonc'] >= 3)
    & (grouped['fonc_1'] == set([0, 9])) | (grouped['fonc_1'] == set([0, 9]))
)

In [246]:
par_nature = grouped['IDENT'].to_frame()
par_nature['vote_par_nature'] = True

In [247]:
par_nature

,IDENT,vote_par_nature
0,20000871200016,True
1,20000872000019,True
2,20000873800011,True
3,20000874600014,True
4,20000875300010,True
...,...,...
5058,21974020600012,True
5059,21974021400016,True
5060,21974022200019,True
5061,21974023000012,True


In [223]:
grouped[conditions].groupby(('fonc_1', 'nunique')).count()

,IDENT,fonc,fonc_1
,,max_length,amax
"(fonc_1, nunique)",,,
2,401,401,401
3,72,72,72
4,18,18,18
5,7,7,7
6,8,8,8
7,22,22,22
8,146,146,146
9,639,639,639


In [190]:
grouped[grouped[('fonc_1', 'nunique')] == 1]

IDENT       fonc  fonc_1     
                     max_length nunique amax
9     20000881100016          3       1    0
14    20000886000013          3       1    0
15    20000887800015          3       1    0
27    20003365200179          3       1    0
30    20003448600049          2       1    0
...              ...        ...     ...  ...
4780  21920036700312          3       1    0
4797  21920073000261          6       1    0
4846  21940002500264          3       1    0
4995  21971129800188          2       1    0
5055  21974017200131          2       1    0

[573 rows x 4 columns]

In [188]:
grouped[conditions]

Empty DataFrame
Columns: [(IDENT, ), (fonc, max_length), (fonc_1, nunique), (fonc_1, amax)]
Index: []

In [178]:
grouped.groupby(('fonc_1', 'nunique')).count()

,IDENT,fonc,fonc_1
,,max_length,amax
"(fonc_1, nunique)",,,
1,573,573,573
2,1205,1205,1205
3,164,164,164
4,44,44,44
5,38,38,38
6,88,88,88
7,151,151,151
8,339,339,339


In [144]:
max_length(data['FONCTION'].apply(str))

1

In [ ]:
# tronquer et étiquetter les codes fonctions
data['FONCTION'].fillna(0, inplace=True)
d = {}
for i in range(len(data["IDENT"])):
    if data["IDENT"][i] in d:
        d[data["IDENT"][i]].add(str(data['FONCTION'][i]))
    else:
        d[data["IDENT"][i]] = set(str(data['FONCTION'][i]))
data["vote_fonction"] = data["IDENT"].apply(lambda x: (len(d[x]) == 1) & (d[x] == set(['9'])))

def f(x, y):
    if y and len(str(x)) >= 3:
        return str(x)[2]
    return str(x)[0]
    
data['code_fonction'] = data.apply(lambda x: f(x.FONCTION, x.vote_fonction, x), axis=1)
data['fonction'] = data['code_fonction'].apply(lambda x: label_fonction(int(x)))

In [43]:
# étiquetter les fonctions
data['FONCTION'].fillna(0, inplace=True)
data['code_fonction'] = data['FONCTION'].apply(lambda x: str(x)[0])
data['fonction'] = data['code_fonction'].apply(lambda x: label_fonction(int(x)))

In [44]:
# concat departement et code insee 
data['departement'] = data['NDEPT'].apply(pad_ndept)
data['code_commune'] = data['departement'] + data['INSEE'].apply(lambda x: str(int(x)))

In [308]:
grouped_data = (
    data
    .groupby(['siren', 'IDENT', 'fonction', 'code_commune', 'aggregat'])
    .agg({'depense': 'sum'})
    .reset_index()
)

In [310]:
# import and join pop data
pop_data = models.population_data()
grouped_data_w_pop = grouped_data.merge(pop_data, how='left', on='code_commune')

In [322]:
grouped_data.head().iloc[0, 3]

'86982'

In [324]:
pop_data.head().iloc[0, 0]

'31555'

In [328]:
grouped_data_w_pop.shape

(254892, 8)

In [327]:
[grouped_data_w_pop['population_2016'].isnull()].shape

(166885, 8)

In [ ]:
grouped_data_w_pop.to_csv('pivot_pop.csv')

In [ ]:
pivot_pop_sample = pivot_pop[pivot_pop['code_commune'].apply(str).isin(['31555', '06088', '44109', '34172'])]

In [ ]:
pivot_pop_sample.to_csv('pivot_pop_sample.csv')

In [336]:
test = pd.read_csv('data_output/bilan_2013.csv')

In [337]:
test[test['population_2016'].isnull()].shape

(166885, 9)

In [338]:
test.shape

(254892, 9)

In [340]:
test[test['commune'] == 'Toulouse']

,Unnamed: 0,siren,IDENT,fonction,code_commune,aggregat,depense,commune,population_2016
44441,44441,213105554,21310555400017,Action économique,31555,Achats et charges externes,318480.89,Toulouse,475442.0
44442,44442,213105554,21310555400017,Action économique,31555,Autres charges de gestion courante,1075523.00,Toulouse,475442.0
44443,44443,213105554,21310555400017,Action économique,31555,Charges de personnel,39403.86,Toulouse,475442.0
44444,44444,213105554,21310555400017,Action économique,31555,Charges exceptionnelles,736499.16,Toulouse,475442.0
44445,44445,213105554,21310555400017,Action économique,31555,Depenses directes d'investissement,882920.75,Toulouse,475442.0
...,...,...,...,...,...,...,...,...,...
44503,44503,213105554,21310555400017,Sport et jeunesse,31555,Subventions d equipement versees,125000.00,Toulouse,475442.0
44504,44504,213105554,21310555400017,Sécurité et salubrité publiques,31555,Achats et charges externes,1064996.05,Toulouse,475442.0
44505,44505,213105554,21310555400017,Sécurité et salubrité publiques,31555,Autres charges de gestion courante,53500.00,Toulouse,475442.0
44506,44506,213105554,21310555400017,Sécurité et salubrité publiques,31555,Charges de personnel,13436769.87,Toulouse,475442.0
